In [2]:
import pandas as pd
import dask
from dask import dataframe as dd
from dask.distributed import Client
import keras
from keras.models import load_model
import librosa
import numpy as np
np.random.seed(1001)

import pickle
import os
import shutil

import IPython
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook
from sklearn.model_selection import StratifiedKFold, train_test_split
import sklearn
import dask.dataframe as dd
from dask import array

import librosa
import IPython.display as ipd  # To play sound in the notebook
import librosa.display

import keras
from keras.layers import Conv1D, Dropout, Dense, MaxPooling1D, Flatten, Conv2D, MaxPooling2D
from keras import Sequential

%matplotlib inline
matplotlib.style.use('ggplot')

import dask
from dask.distributed import Client, wait, progress
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


# Todo



In [5]:
res = dd.read_parquet("res/*.parquet").compute()

# Best model params (2D) 

In [6]:
md = load_model("md/0.1-2-64-128-False.md")

In [7]:
pd.read_parquet("res/0.1-2-64-128-False.parquet")

,dropout,n_conv,base_dense,n_filters,loss,acc
5,0.1,2.0,64.0,128.0,0.169319,0.1


# Best 1D model

In [8]:
# load it

# Preprocessing

In [9]:
def get_stft_2d(y, sr = 22050, hop_length=32):
    D = librosa.stft(y, hop_length=hop_length, n_fft=1024)
    spec = librosa.amplitude_to_db(D,ref=np.max)
    df = pd.DataFrame(spec / 80)
    return spec

In [10]:
def get_length_in_seconds(y,sr):
    return len(y)/sr

def create_samples_from_fn(fn, desired_sr = 22050):
    """
    Also trims silence from a file
    """    
    try:
        # Load, resample if needed
        y, sr = librosa.load(fn)
    except:
        return None
    if sr != desired_sr:
        y = librosa.core.resample(y, sr, desired_sr)
        sr = desired_sr

    # Standard scaling
    standardScaler = StandardScaler()    
    y = standardScaler.fit_transform(y.reshape(-1, 1)).reshape(1, -1)[0]

    length = get_length_in_seconds(y, sr)

    # Trim silence
    y_trimmed = librosa.effects.trim(y, top_db=12.5)[0]
    length_trimmed = get_length_in_seconds(y_trimmed, sr)

    # Split into chunks
    chunk_len = int(sr / 2) # .5 seconds
    end = len(y_trimmed) - (len(y_trimmed) % chunk_len)
    n_chunks = int(end / chunk_len)
    if n_chunks == 0:
        return None
    y_trimmed_chunks = np.split(y_trimmed[:end], n_chunks)
    y_trimmed_chunks = [x for x in y_trimmed_chunks]
    
    return y_trimmed_chunks

In [11]:
def get_ts_features(y,sr = 22050, hop_length=32):
    n_cont_bands = 2
    res = pd.DataFrame(columns=["contrast_band_{}".format(x) for x in range(n_cont_bands+1)].extend(["sroll", "sflat"]))
    
    scont = librosa.feature.spectral_contrast(y,n_bands=n_cont_bands, hop_length=hop_length)
    
    for i in range(n_cont_bands + 1):
        res["contrast_band_{}".format(i)] = scont[i]
        
    sroll = librosa.feature.spectral_rolloff(y,sr, hop_length=hop_length)[0].T
    res["sroll"] = sroll
    
    sflat = librosa.feature.spectral_flatness(y, hop_length=hop_length)[0].T
    res["sflat"] = sflat
    
    y_df = pd.DataFrame(y)
    res["downsampled"] = y_df.groupby(y_df.index//hop_length).mean().rolling(2).mean()
    res["downsampled_smooth_abs"] = y_df.abs().groupby(y_df.index//hop_length).mean().rolling(6).mean()

    return res.fillna(method="backfill").fillna(method="ffill")

In [12]:
def preprocess_sample(fn):
    # Load
    chunks = create_samples_from_fn(fn)
    # Break into chunks
    print(chunks)
    
    # 2D transformation
    chunks_2d = [get_stft_2d(ch) for ch in chunks]
    
    # 1D transformation
    chunks_1d = [get_ts_features(ch).values for ch in chunks] 
    print(chunks_2d)
    print(chunks_1d)
    return chunks_1d, chunks_2d 
    
one, two = preprocess_sample("../data/listenr-ml/RAVDESS-Audio_Speech_Actors_01-24/Actor_02/03-01-01-01-01-01-02.wav")
    

[array([-0.3588106 , -0.00266513,  0.44744202, ..., -0.37154463,
       -0.49179968, -0.34090018], dtype=float32), array([-0.02592736,  0.06822344, -0.03271784, ..., -0.9426107 ,
       -0.9915499 , -1.0199014 ], dtype=float32)]


c:\users\chris\appdata\local\programs\python\python35\lib\site-packages\librosa\core\spectrum.py:960: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  warnings.warn('amplitude_to_db was called on complex input so phase '


[array([[-54.67341 , -54.690697, -54.840923, ..., -35.32738 , -34.876953,
        -34.654335],
       [-57.42822 , -57.548817, -57.774273, ..., -35.23271 , -35.01561 ,
        -34.909195],
       [-55.891888, -55.953175, -56.233017, ..., -36.299625, -35.831867,
        -35.59884 ],
       ...,
       [-76.11128 , -76.197815, -76.4498  , ..., -80.      , -80.      ,
        -80.      ],
       [-76.38053 , -76.46394 , -76.727394, ..., -80.      , -80.      ,
        -80.      ],
       [-76.38255 , -76.4669  , -76.70944 , ..., -80.      , -80.      ,
        -80.      ]], dtype=float32), array([[-32.049377, -32.097584, -32.351387, ..., -37.92804 , -37.538555,
        -37.572678],
       [-32.571735, -32.687756, -32.92295 , ..., -37.288055, -36.94465 ,
        -36.576015],
       [-35.860558, -35.850716, -35.98919 , ..., -34.803413, -34.509445,
        -34.501366],
       ...,
       [-67.725845, -67.8069  , -68.056656, ..., -80.      , -80.      ,
        -80.      ],
       [-67.783485

In [3]:
md_2d = load_model("md/0.1-2-64-128-False.md")
md_1d = load_model("md/1d.model")

In [16]:
one[0].shape

(345, 7)

In [15]:
two[0].shape

(513, 345)

In [18]:
md_2d.predict(two[0].reshape(-1, 513, 345, 1))

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_1/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_1/kernel/read)]]